In [13]:
import numpy as np

In [14]:
#define code property
m = 12
l = 6

# define generation set
a_pows = [(3, 0), (0, 1), (0, 2)]
b_pows = [(0, 3), (1, 0), (2, 0)]

# define logical operator X
f = [(0, 0), (1, 0), (2, 0), (3, 0), (6, 0), (7, 0), (8, 0), (9, 0), (1, 3), (5, 3), (7, 3), (11, 3)]
g = [(1, 0), (2, 1), (0, 2), (1, 2), (2, 3), (0, 4)]
h = [(0, 0), (0, 1), (0, 2), (0, 3), (1, 1), (1, 3)]

#dag operation
def dag(a_pows: list[tuple[int, int]], t1=m, t2=l):
    b_pows = []
    for i in a_pows:
        b_pows.append((t1-i[0], t2-i[1]))
    return b_pows

hdag = dag(h)
xset = [(0, 0), (0, 1), (2, 1), (2, 5), (3, 2), (4, 0)]
zset = [(0, 1), (0, 5), (1, 1), (0, 0), (4, 0), (5, 2)]
fix = 3


In [15]:
# Define commute operation
def is_commute(a_pows: list[tuple[int, int]], b_pows: list[tuple[int, int]]):
    k = 0
    for i in range(len(a_pows)):
        for j in range(len(b_pows)):
            if a_pows[i] == b_pows[j]:
                k = k+1
    if k % 2 == 0:
        return 1
    else:
        return 0


# Define automorphism multiplication
def multione(a_pows: list[tuple[int, int]], omega: tuple[int, int], t1=m, t2=l):
    b_pows = []
    for i in a_pows:
        b_pows.append(((i[0] + omega[0]) % t1, (i[1] + omega[1]) % t2))
    return b_pows

# def multitwolist(am: list[tuple[int, int]], bm: list[tuple[int, int]],t1=m,t2=l):
#     cm=[]
#     for i in len(am):
#         cm.append(((am[i][0]+ bm[i][0]) % t1, (am[i][1]+ bm[i][1]) % t2))
#     return cm





# Automorphism generation set
gen = []
for i in a_pows:
    for j in dag(a_pows):
        gen.append(((i[0]+j[0]) % m, (i[1]+j[1]) % l))
for i in b_pows:
    for j in dag(b_pows):
        gen.append(((i[0]+j[0]) % m, (i[1]+j[1]) % l))
gen = list(set(gen))

# compute the set of achievable monomial
def generation(a_pows):
    cm = []
    for i in a_pows:
        for j in gen:
            cm.append(((i[0]+j[0]) % m, (i[1]+j[1]) % l))
    cm = list(set(cm))
    return cm

gen1 = gen
genhe =[]
genhe.append([(0,0)])
genhe.append(gen)
for i in range(5):
    gen1= generation(gen1)
    genhe.append(gen1)





In [16]:
#find monomial for multi Z automotphism
def findomega(target):
    omega=[]
    for t1 in range(m):
        for t2 in range(l):
            test_omega = (t1,t2)
            comm = [1] * 6
            for j in range(len(xset)):
                if is_commute(multione(hdag, zset[0]), multione(multione(f, xset[j]),test_omega)):
                    comm[j] = 0
            if comm == target: 
                omega.append(test_omega)
    if omega == []:
        target1 = target 
        target1[fix] = 1
        for t1 in range(m):
            for t2 in range(l):
                test_omega = (t1,t2)
                comm = [1] * 6
                for j in range(len(xset)):
                    if is_commute(multione(hdag, zset[0]), multione(multione(f, xset[j]),test_omega)):
                        comm[j] = 0
                if comm == target1: 
                    omega.append(test_omega)
        if omega == []:
            print(target1,"error")
        else:
            print(target1,"fixed")
    return omega

#find minimum round to synthesize monomial
def number(omega):
    c = 1000
    for i in omega:
        for j in range(len(genhe)):
            for k in genhe[j]:
                if (k == i)&(c > j):
                    c = j
    return c
                
def totalnum(protocal):
    total = 0
    for i in protocal:
        total += number(findomega(i))
    return total






In [17]:
#define protocal circuits
fiftn = [[0, 1, 0, 0, 0, 0],[0 ,0, 1, 0, 0, 0],[0 ,0, 0, 1, 0, 0],[0 ,0, 0, 0, 1, 0],
         [0, 1, 1, 1, 0, 0],[1, 1, 1, 0, 0, 0],[1, 1, 0, 1, 0, 0],[1, 0, 1, 1, 0, 0],[1, 0, 0, 1, 1, 0],
         [1, 1, 0, 0, 1, 0],[1, 0, 1, 0, 1, 0],[1, 1, 1, 1, 1, 0],[0, 0, 1, 1, 1, 0],[0, 1, 0, 1, 1, 0],[0, 1, 1, 0, 1, 0]]


for i in fiftn:
    i[5] = i[fix]
    i[fix] = 0

twenty = [[0, 1, 0, 1, 0, 0],[0, 1, 0, 1, 0, 0],[0, 1, 0, 1, 0, 0],[0, 1, 0, 1, 0, 0],[0, 1, 0, 1, 0, 0],[0, 1, 0, 1, 0, 0]]


print(totalnum(fiftn))







[1, 1, 1, 1, 0, 0] fixed
[1, 1, 0, 1, 0, 1] fixed
[1, 0, 0, 1, 1, 1] fixed
[1, 0, 1, 1, 1, 0] fixed
26


In [18]:
# for t1 in range(m):
#         for t2 in range(l):
#             test_omega = (t1,t2)
#             comm = [1] * 6
#             for j in range(len(xset)):
#                 if is_commute(multione(hdag, zset[0]), multione(multione(f, xset[j]),test_omega)):
#                     comm[j] = 0
#             print(test_omega,comm)